In [1]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from PyPDF2 import PdfReader
from transformers import pipeline

import chromadb
import torch
import ollama

/home/nayra/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
PATH = "pdf/JavaBasico.pdf"


reader = PdfReader(PATH)

text_load = ''.join(page.extract_text() for page in reader.pages)

client = chromadb.PersistentClient(path="chromadb")
collection = client.get_or_create_collection(name="vector-langchain")
history_collection = client.get_or_create_collection(name="chat_history")
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

'Volume único\nJava Básico e \nOrientação a ObjetoClayton Escouper das Chagas\nCássia Blondet Baruque\nLúcia Blondet BaruqueClayton Escouper das Chagas\nCássia Blondet BaruqueLúcia Blondet BaruqueVolume únicoJava Básico e \nOrientação a Objeto\nApoio:\n2010/1Material Didático\nORGANIZAÇÃO\nCássia Blondet Baruque\nELABORAÇÃO DE CONTEÚDO\nClayton Escouper das Chagas\nCássia Blondet BaruqueLúcia Blondet BaruqueFundação Cecierj / Extensão\nRua Visconde de Niterói, 1364 – Mangueira – Rio de Janeiro, RJ – CEP 20943-001\nTel.: (21) 2334-1569 Fax: (21) 2568-0725\nPresidente\nMasako Oya Masuda\nVice-presidente\nMirian Crapez\nCoordenadoras da Área de Governança: Gestão, Auditoria \ne Tecnologia da Informação\nLúcia Blondet Baruque\nCássia Blondet Baruque\nEDITORA\nTereza Queiroz\nCOPIDESQUE\nCristina Freixinho\nREVISÃO TIPOGRÁFICA\nElaine BaymaDaniela SouzaJanaína SantanaThelenayce Ribeiro\nCOORDENAÇÃO DE PRODUÇÃO\nKaty AraujoPROGRAMAÇÃO VISUAL\nMárcia Valéria de AlmeidaRonaldo d´Aguiar Silva\n

In [3]:
chunks = []  
start_index = 0
chunk_size = 1024
chunk_overlap = 20

while start_index < len(text_load):
    end_index = start_index + chunk_size  
    chunks.append(text_load[start_index:end_index])
    start_index = end_index - chunk_overlap
embeddings_db = embeddings.encode(chunks)

In [4]:
for i, d in enumerate(chunks):

      collection.add(
              ids=[str(i)],
              embeddings=embeddings_db[[i]],
              documents=[d]
    )

In [6]:
while True:

    prompt = input("Pergunte algo")
    response = embeddings.encode([prompt])
    results = collection.query(
            query_embeddings=response,
            n_results=2
        )

    data = results['documents'][0]
    distance = results['distances']

    print(distance)
    print(data)
    
    for k in range(2):
    
        if distance[0][k] > 0.9:
            data = "Você é um assistente de chatbot e tem acesso limitado a informações. Nesse caso, diga que não tem conhecimento acerca da pergunta."

    output = ollama.generate(
            model="llama2",
            prompt=f"""
            Você é um chatbot assistente.
            Com base na frase do usuário: {prompt}, crie um quiz com duas perguntas tirando o contexto somente dos seguinte dados: {data}.
            """
    )
    output = output['response']
    print(output)
    print("\n")

Pergunte algo orientação a objeto


[[0.4426756501197815, 0.6076918840408325]]
['o e Orientação a Objeto  | Orientação a Objeto I\nAlgumas observações de boas práticas:\n\x81  As declarações dos atributos são feitas sempre na primeira parte \ndo código da classe.\n\x81  Declare um atributo por linha, mesmo que ele seja do mesmo \ntipo.\n\x81  Declare atributo como private ; assim, somente a própria classe \npode manipulá-lo; esse é o conceito de encapsulamento , muito \nimportante na Orientação a Objeto.\nAlém dos atributos do objeto, que será individual para cada \naluno, podemos declarar também um atributo que será compartilhado por todos os objetos daquele tipo. Na verdade, o que criamos é o que chamamos de atributo de classe . Isso é feito por meio da palavra static  \nantes da deﬁ  nição do atributo. No nosso exemplo, um bom atributo de \nclasse pode ser aquele que vai contar a quantidade de alunos, o conta-\ndorEstudante :\nAs classes são compostas por atributos e métodos. O primeiro já \nresolvemos; vamos agora im

Pergunte algo classe 


[[1.0501177310943604, 1.2619414329528809]]
[' carro de forma geral, você está aprendendo um conceito bastante importante em Java: o conceito de classe. \nClasse é a deﬁ  nição \nde um conjunto de objetos, seres ou \ncoisas do mundo real que possuem\ncaracterísticas  em comum. \nTodo programa em Java está dentro de uma estru-\ntura lógica de código chamada classe; portanto, um\nsistema em Java é um con junto de classes que se \ncomunicam e colabo ram entre si.\nAsclasses são compostas de atributo s\nemétodo s.\nNo nível de análise e requisitos, os atributos  são as características \nque os objetos do mundo real possuem, e estes são sua representação no mundo computacional.\nFazendo a mesma comparação, os métodos  são os comportamen-\ntos ou as ações executadas por esses objetos.\nAssim, com um exemplo bem simples, podemos dizer que, se \nestamos deﬁ  nindo e programando o objeto do mundo real carro , a classe \nrelativa a esse objeto terá atributos como cor, modelo, ano de fabricação \n

KeyboardInterrupt: Interrupted by user